# Comparação de Modelos

## Previsões

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv('./dados/dados_2.csv') #Lendo os dados

In [2]:
#Declarando as variáveis:
X = df[['Radiacao (KJ/m²)', 'Temp. Ins. (C)', 'Umi. Ins. (%)', 'Chuva (mm)', 'Hora', 'Mês', 'Pto Orvalho Ins. (C)', 'Temp. Max. (C)',
       'Temp. Min. (C)']]

y = df['Potência CA (W)']


#Separando em treino e teste:
X_t, X_test, y_t, y_test = train_test_split(X, y, test_size=0.1, random_state=101) #10% dos dados são p/ teste

### SVR

In [4]:
from sklearn.svm import SVR
reg = SVR(C=40, epsilon=0.1)

#Normalizando os dados de entrada:
scaler = StandardScaler()
X_t_scaled = scaler.fit_transform(X_t)
X_test_scaled = scaler.fit_transform(X_test)

y_t_log = np.log(y_t) #Aplicando transformação log nos dados de saída

#Treinando o modelo:
reg.fit(X_t_scaled, y_t_log)

#Fazendo a previsão:
pred_log = reg.predict(X_test_scaled)
svr = np.exp(pred_log)

### Árvore de Decisão

In [7]:
from sklearn.tree import DecisionTreeRegressor
reg = DecisionTreeRegressor(min_samples_leaf=16)

reg.fit(X_t, y_t)

dt = reg.predict(X_test)

### KNR

In [8]:
from sklearn.neighbors import KNeighborsRegressor
reg = KNeighborsRegressor(n_neighbors=9,weights='distance', p=2)

#Normalizando os dados de entrada:
scaler = StandardScaler()
X_t_scaled = scaler.fit_transform(X_t)
X_test_scaled = scaler.fit_transform(X_test)


#Treinando o modelo:
reg.fit(X_t_scaled, y_t)

#Fazendo a previsão:
knr = reg.predict(X_test_scaled)

### RNA

In [9]:
from sklearn.neural_network import MLPRegressor
layers = (50,50,50,50,50)
reg = MLPRegressor(hidden_layer_sizes = layers, activation='relu', solver='lbfgs', learning_rate='adaptive', random_state=101)

#Normalizando os dados de entrada:
scaler = StandardScaler()
X_t_scaled = scaler.fit_transform(X_t)
X_test_scaled = scaler.fit_transform(X_test)


#Treinando o modelo:
reg.fit(X_t_scaled, y_t)

#Fazendo a previsão:
rna = reg.predict(X_test_scaled)

## Teste de Dunnett